In [1]:
!pip install pydantic-ai


  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 11.0 MB/s eta 0:00:00
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: groq
    Found existing installation: groq 0.11.0
    Uninstalling groq-0.11.0:
      Successfully uninstalled groq-0.11.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.28.1 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
unstructured-client 0.28.1 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.4 which is incompatible.


In [21]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o', system_prompt='Be a helpful assistant.')



In [23]:
# import nest_asyncio
# nest_asyncio.apply()

In [24]:
# result = agent.run_sync('Tell me a joke.')
result = agent.run_sync('Tell me a joke.')

In [25]:
result


RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')], _new_message_index=0, data="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", _result_tool_name=None, _usage=Usage(requests=1, request_tokens=21, response_tokens=18, total_tokens=39, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0}))

In [31]:
#> Did you hear about the toothpaste scandal? They called it Colgate.

# all messages from the run
print(result.new_messages())

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')]


In [34]:
result.all_messages()


[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')]

In [33]:
result.data


"Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!"

## roulette_wheel

In [36]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(  
    'openai:gpt-4o',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:  # ctx is the context of the run
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'


# Run the agent
success_number = 18  
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(result.data)  
#> True

result = roulette_agent.run_sync('I bet five is the winner', deps=success_number)
print(result.data)
#> False

True
False


## run_agent